In [9]:
#Many columns of the table have categorical data
#We use One-hot econding schema to convert this data to binary
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv("recs2009_public.csv",index_col=0)
enc = OneHotEncoder(handle_unknown='ignore')

#There are many columns showing parameters like "temperature when...", but showing -2 when not aplicable.
#We need to separate this type of data into a binary (not aplicable or aplicable) and a non-categorical column
#In the non-categorical column, we will replace "-2" with the average of all the values different from "-2"

def seriesIntoBinaryAndNonCateg(s,valuesToBinary):
    mean = s[s > 0].mean()
    nonCategSeries = s.replace(valuesToBinary,mean)
    categSeriesList = [(s == value).astype(int) for value in valuesToBinary]
    return pd.concat([nonCategSeries]+categSeriesList,axis=1)

cols_categ_with_binary = {21,22,26,29,38,41,44,48,54,146,152,310,460,462,466,467,468,540,546,547,548,600,602,716,723,776}

list_all_noncateg_and_binary = list()
for col in cols_categ_with_binary:
    #I found columns with "." that I am assuming to mean "again -2"
    s = pd.to_numeric(df[df.columns[col]].replace('.',-2))
    list_all_noncateg_and_binary.append(seriesIntoBinaryAndNonCateg(s,[-2]))

df_all_noncateg_and_binary = pd.concat(list_all_noncateg_and_binary,axis=1)

#In the case of column "NKRGALNC", 77 means "not sure". thus we have values -2 and 77 to trasnform to binary
#And a non-categorical integer
#in the same way 

s = pd.to_numeric(df[df.columns[717]].replace('.',-2)) #asumming "." is "-2" to save time
df_717 = seriesIntoBinaryAndNonCateg(s,[-2,77])

#In a similar way, columns 595 597 599 601, can be trasnform into a non-categorical column and 3 binary columns 
#corresponding to values -2, -8, -9

cols_noncateg_and_3_binaries = {595,597,599,601}
list_all_noncateg_and_3_binaries = list()
for col in cols_noncateg_and_3_binaries:
    s = pd.to_numeric(df[df.columns[col]].replace('.',-2)) #asumming "." is "-2" to save time
    list_all_noncateg_and_3_binaries.append(seriesIntoBinaryAndNonCateg(s,[-2,-8,-9]))

df_all_noncateg_and_3_binaries = pd.concat(list_all_noncateg_and_3_binaries,axis=1)
    
# We create a list of fully non-categorical columns, as most columns are categorical
cols_full_noncateg=set([4,5,6,7,8,30,31,32,33,113,115,117,133,238,288,502,503,594,596,598,758,759,784] 
                    +list(range(826,836))+list(range(856,918))+list(range(931,939)))
df_full_noncateg=df[df.columns[list(cols_full_noncateg)]]
# The gloal is predicting electricity usage from residential units so we remove all columns that
# give direct information about electricity usage, and electricity cost 'KWHSPH'...'DOLELRFG'
cols_to_ignore = set(range(839,855))

#the raminig columns correspond to the full categorical ones
cols_full_categ = [col for col in range(len(df.columns)) if col not in cols_categ_with_binary \
                     and col not in cols_full_noncateg and col not in cols_noncateg_and_3_binaries \
                     and col != 717 and col not in cols_to_ignore and col != 838] #


df_categorical = df[df.columns[cols_full_categ]]
X = df_categorical.to_numpy().tolist()
enc.fit(X)
Y = enc.transform(X).toarray()
df_binary = pd.concat([pd.DataFrame(Y),pd.DataFrame(df.index)]).set_index("DOEID")


df_encoded = pd.concat([df_all_noncateg_and_binary,df_717,df_all_noncateg_and_3_binaries,df_binary,df_full_noncateg],axis=1)
print(df_encoded)



/miniconda/envs/Zesty/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (717,718) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


        TELLDAYS  TELLDAYS   MONRFRI2  MONRFRI2   NUMFLRS  NUMFLRS   NUMAPTS  \
DOEID                                                                          
1.0    10.573346       1.0  12.000000       0.0  3.864275      1.0  38.75871   
2.0    10.573346       1.0  11.111927       1.0  3.864275      1.0  38.75871   
3.0    10.573346       1.0  11.111927       1.0  2.000000      0.0   8.00000   
4.0    10.573346       1.0  12.000000       0.0  3.864275      1.0  38.75871   
5.0    10.573346       1.0  11.111927       1.0  3.864275      1.0  38.75871   
...          ...       ...        ...       ...       ...      ...       ...   
NaN          NaN       NaN        NaN       NaN       NaN      NaN       NaN   
NaN          NaN       NaN        NaN       NaN       NaN      NaN       NaN   
NaN          NaN       NaN        NaN       NaN       NaN      NaN       NaN   
NaN          NaN       NaN        NaN       NaN       NaN      NaN       NaN   
NaN          NaN       NaN        NaN   

In [80]:
for col in cols_categ_with_binary:
    s = pd.to_numeric(df[df.columns[[col]][0]].replace('.',-2))
    print(s[s < 0].dropna().unique())
    try:
        print(df.columns[[col]],col, s[s < 0].dropna().unique())
    except:
        print(df.columns[[col]],col)
        
p = pd.to_numeric(df[df.columns[[717]]][df.columns[[717]][0]].replace('.',"-2")) 
#for i in range()
print(p.to_list())

[-2]
Index(['NUMFLRS'], dtype='object') 22 [-2]
[-2]
Index(['NUMAPTS'], dtype='object') 23 [-2]
[-2]
Index(['NAPTFLRS'], dtype='object') 27 [-2]
[-2]
Index(['BEDROOMS'], dtype='object') 30 [-2]
[-2]
Index(['FINBASERMS'], dtype='object') 39 [-2]
[-2]
Index(['PCTBSTHT'], dtype='object') 42 [-2]
[-2]
Index(['PCTBSTCL'], dtype='object') 45 [-2]
[-2]
Index(['FINATTRMS'], dtype='object') 49 [-2]
[-2]
Index(['PCTATTCL'], dtype='object') 55 [-2]
[-2]
Index(['MONRFRI2'], dtype='object') 147 [-2]
[-2]
Index(['MONRFRI3'], dtype='object') 153 [-2]
[-2]
Index(['PCPRINT'], dtype='object') 311 [-2]
[-2]
Index(['HEATROOM'], dtype='object') 461 [-2]
[-2]
Index(['NUMTHERM'], dtype='object') 463 [-2]
[-2]
Index(['TEMPHOME'], dtype='object') 467 [-2]
[-2]
Index(['TEMPGONE'], dtype='object') 468 [-2]
[-2]
Index(['TEMPNITE'], dtype='object') 469 [-2]
[-2]
Index(['ACROOMS'], dtype='object') 541 [-2]
[-2]
Index(['TEMPHOMEAC'], dtype='object') 547 [-2]
[-2]
Index(['TEMPGONEAC'], dtype='object') 548 [-2]
[-2]
I

,DOEID
0,1
1,2
2,3
3,4
4,5
...,...
12078,12079
12079,12080
12080,12081
12081,12082


In [ ]:
#we then create a training and a test set
#we will only use the test set when the models are complete in order to validate it

import pandas as pd
import numpy as np


msk = np.random.rand(len(df)) < 0.8
df_training = df[msk]
df_test = df[~msk]

In [22]:
#Because of the nature of the data, I expect it to be stochastic to some extent. Thus, I start using random forest because of its simplicity, and then MLP classifier with stochastic gradient descent.
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


#We now create dataframes for the input and output data of the model
#I am assuming all other columns can be used as input, for simplicity.
df_X = df_training.drop(["KWH"],axis =1) 
df_Y = df_training["KWH"]
clf = RandomForestClassifier(max_depth=2, random_state=0)
print(list(df_X.values)[0])
#clf.fit(list(df_X.values), list(df_Y.values))


[1 2 4 12 2 2471.68 4742 1080 4953 1271 4 3 'METRO' 'U' 1 -2 2004 7 8 -2
 -2 -2 -2 -2 1 5 -2 -2 20 -2 4 1 2 5 9 1 0 0 1 1 1 2 3 1 2 3 1 0 -2 -2 -2
 -2 -2 -2 -2 -2 -2 1 2 2 0 0 -2 -2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 5 0 -2 0 -2
 3 1 1 1 3 1 1 2 0 -2 0 4 5 1 2 21 4 2 1 3 1 -2 -2 -2 21 4 2 12 3 1 -2 -2
 -2 -2 -2 -2 0 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 1 13 3 1 -2 -2 -2 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 2 2 3 3 2 1 1 0 -2 1 5 1 1 4 3 2 1 1 -2 0 1 0 0 1 1 0
 3 4 3 4 3 4 1 1 -2 0 0 0 0 0 0 0 2 -2 2 -2 3 3 0 -2 0 0 0 0 0 1 0 0 3 -2
 2 -2 1 2 1 1 3 0 1 1 1 3 0 1 -2 -2 -2 -2 -2 1 0 1 0 0 1 1 0 0 0 -2 -2 0 0
 1 1 1 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 -2 -2
 -2 3 5 0 3 -2 -2 -2 0 0 0 0 -2 

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
print(y)
clf.fit(X, y)
RandomForestClassifier(...)
print(clf.predict([[0, 0, 0, 0]]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 